# Eat Safe, Love

## Notebook Set Up

In [4]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our database


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [13]:
# Find the establishments with a hygiene score of 20
hygiene_20_query ={"scores.Hygiene": 20}
hygiene_20_establishments = establishments.find(hygiene_20_query)

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(hygiene_20_query)

# Display the first document in the results using pprint
first_hygiene_20 = hygiene_20_establishments[0]
pprint(first_hygiene_20)


IndexError: no such item for Cursor instance

In [14]:
# Convert the result to a Pandas DataFrame
import pandas as pd

# Display the number of rows in the DataFrame
hygiene_20_df = pd.DataFrame(list(hygiene_20_establishments))

# Display the first 10 rows of the DataFrame
print(hygiene_20_df.head(10))

Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [15]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
london_rating_query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Case-insensitive regex for "London"
    "RatingValue": {"$gte": 4}
}
london_rating_establishments = establishments.find(london_rating_query)

# Use count_documents to display the number of documents in the result
london_rating_count = establishments.count_documents(london_rating_query)

# Display the first document in the results using pprint
first_london_rating = next(london_rating_establishments)
pprint(first_london_rating)

StopIteration: 

In [16]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(list(london_rating_establishments))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {london_rating_df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(london_rating_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [17]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = penang_flavours_coords["latitude"]
longitude = penang_flavours_coords["longitude"]

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("scores.Hygiene", 1)] 
limit = 5

# Print the results
for result in results:
    pprint(result)

NameError: name 'penang_flavours_coords' is not defined

In [18]:
# Convert result to Pandas DataFrame
results_list = list(results)
df = pd.DataFrame(results_list)

print(df.head())

NameError: name 'results' is not defined

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [19]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    
    {"$sort": {"count": -1}}
]

result = establishments.aggregate(pipeline)
result_list = list(result)

# Print the number of documents in the result
print(f"Number of Local Authorities with hygiene score of 0: {len(result_list)}")

# Print the first 10 results
pprint(result_list[:10])

Number of Local Authorities with hygiene score of 0: 0
[]


In [20]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []
